In [76]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [77]:
import csv

In [78]:
data = []
with open('data/E0 (20).csv', 'r', encoding='utf-16') as f:
    reader = csv.reader(f)
    for row in reader:
        data.append(row[: 26])
        
data = pd.DataFrame(np.array(data))

In [79]:
data

,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
0,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,AF,HC,AC,HY,AY,HR,AR,B365H,B365D,B365A
1,E0,14/08/04,Aston Villa,Southampton,2,0,H,2,0,H,...,9,12,6,0,0,0,0,1.9,3.1,4.33
2,E0,14/08/04,Blackburn,West Brom,1,1,D,0,1,A,...,17,4,5,1,0,0,0,1.8,3.1,5
3,E0,14/08/04,Bolton,Charlton,4,1,H,2,0,H,...,12,9,5,1,1,0,0,2.25,3.2,3.1
4,E0,14/08/04,Man City,Fulham,1,1,D,1,0,H,...,12,9,4,0,2,0,0,2.1,3.25,3.4
5,E0,14/08/04,Middlesbrough,Newcastle,2,2,D,0,1,A,...,13,6,7,3,1,0,0,2.3,3.25,3
6,E0,14/08/04,Norwich,Crystal Palace,1,1,D,1,0,H,...,16,6,11,0,1,0,0,2.1,3.25,3.4
7,E0,14/08/04,Portsmouth,Birmingham,1,1,D,1,1,D,...,16,8,4,2,4,0,0,2.4,3.2,2.87
8,E0,14/08/04,Tottenham,Liverpool,1,1,D,0,1,A,...,11,3,8,3,1,0,0,3.1,3.2,2.25
9,E0,15/08/04,Chelsea,Man United,1,0,H,1,0,H,...,8,2,3,0,0,0,0,2.1,3.25,3.4


In [80]:
fixture = []
x = 0
size = 10

for i in range(data.shape[0]):
    d = data[x:size]
    for index, row in d.iterrows():
        if (row[2] == 'Liverpool' or row[3] == 'Liverpool') and (row[2] == 'Aston Villa' or row[3] == 'Aston Villa'):
            fixture.append(row) 
    
    x += 10
    size += 10


fixture = pd.DataFrame(fixture)

In [81]:
fixture

,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
151,E0,04/12/04,Aston Villa,Liverpool,1,1,D,1,1,D,...,10,3,9,1,0,0,0,2.5,3.2,2.75
375,E0,15/05/05,Liverpool,Aston Villa,2,1,H,2,0,H,...,16,9,5,0,0,0,0,1.66,3.4,5.5


In [82]:
fixtures = []
x = 0
size = 10

for i in range(data.shape[0]):
    d = data[x:size]
    for index, row in d.iterrows():
        if (row[2] == 'Liverpool' or row[3] == 'Liverpool'):
            if row[2] == 'Liverpool':
                row.drop([3, 5, 6, 8, 9, 10, 12, 14, 16, 18, 20, 22, 23, 24, 25], inplace=True)
                row = [x for x in row if x != 'NaN']
                fixtures.append(row) 
                
            elif row[3] == 'Liverpool':
                row.drop([2, 4, 6, 7, 9, 10, 11, 13, 15, 17, 19, 21, 23, 24, 25], inplace=True)
                row = [x for x in row if x != 'NaN']
                fixtures.append(row) 
            
    x += 10
    size += 10


fixtures = pd.DataFrame(fixtures)

In [83]:
fixtures

,0,1,2,3,4,5,6,7,8,9,10
0,E0,14/08/04,Liverpool,1,1,16,8,11,8,1,0
1,E0,21/08/04,Liverpool,2,0,15,10,9,8,1,0
2,E0,29/08/04,Liverpool,0,0,7,2,14,2,0,0
3,E0,11/09/04,Liverpool,3,2,16,9,9,8,0,0
4,E0,20/09/04,Liverpool,1,0,6,3,15,2,2,0
5,E0,25/09/04,Liverpool,3,2,20,12,10,4,1,0
6,E0,03/10/04,Liverpool,0,0,5,3,20,2,1,0
7,E0,16/10/04,Liverpool,4,0,10,3,14,6,3,1
8,E0,23/10/04,Liverpool,2,0,17,7,9,12,0,0
9,E0,30/10/04,Liverpool,2,1,13,9,11,6,1,0


In [98]:
fixtures = []

x = 0
size = 10

for i in range(data.shape[0]):
    d = data[x:size]
    for index, row in d.iterrows():
        
        record = np.empty(9, dtype=object)
        
        if (row[2] == 'Liverpool' or row[3] == 'Liverpool'):
            
            row = row.tolist()
            record[0] = row[1].split('/')[1]
            
            if row[2] == 'Liverpool':
                record[1] = 1
                
                if row[9] == 'A': record[4] = 0 
                elif row[9] == 'H': record[4] = 3
                else: record[4] = 1
                    
            elif row[3] == 'Liverpool':
                record[1] = 0
                
                if row[9] == 'A': record[4] = 3 
                elif row[9] == 'H': record[4] = 0
                else: record[4] = 1
            
            record[2] = row[7]
            record[3] = row[8]
            record[5] = row[23]
            record[6] = row[24]
            record[7] = row[25]
            record[8] = row[4]
            
            fixtures.append(record)
            
            
    x += 10
    size += 10


fixtures = pd.DataFrame(fixtures)

In [99]:
fixtures

,0,1,2,3,4,5,6,7,8
0,08,0,0,1,3,3.1,3.2,2.25,1
1,08,1,0,1,0,1.57,3.5,6.5,2
2,08,0,1,0,0,3.2,3.25,2.2,1
3,09,1,2,0,3,1.444,3.75,8.5,3
4,09,0,1,0,0,2,3.2,3.75,2
5,09,1,2,0,3,1.4,4,9,3
6,10,0,0,0,1,1.8,3.1,5,1
7,10,0,2,0,0,3.4,3.25,2.1,2
8,10,1,0,0,1,1.5,3.6,7.5,2
9,10,0,2,1,0,5,3.1,1.8,2
